In [8]:
#Date:10/18/2022
#Author: Talia Kurtz

#Purpose of the Code:

#This code is used to take the CSV file containing the clear day data provided by Lauren Vocke and run it through the NetCdf file that contains the ARM data/SOM winning Node data.
#When the clear day dates are run through the NetCdf file then we will be creating a NetCDF file that contains only the clear day data. The coordinate will be time to keep it
#constant with the ARM/SOM Winning Node data.

#What is saved?
#This code saves the clear day data in a NetCDF file for use later to determine commonalities in the clear day weather patterns.


In [9]:
#Imports
import pandas as pd
import xarray as xr
import csv
import numpy as np
import datetime

In [10]:
#Opening the dataset and stating the path for the files that will be opened later in the code.
PATH = '/Users/research/thesis_code/ARM_Code_Steps/'
 
dx = xr.open_dataset(PATH+'ARM_SOM_2003_2022.nc')
keys = list(dx.keys())

In [12]:
#Opening the text file that contains the clear day data that has been reformatted in a previous notebook and reading it in line by line to make a list.
with open((PATH + 'clear_day_list_hrs.txt') ) as file:
    lines = file.readlines()
    clear_days = [line.rstrip() for line in lines]

In [14]:
#Checking that the dates match the text file
len(clear_days)

3083

In [15]:
#Generating empty lists to store the values that correspond to the Clear Day Data. 
times = []
node_data = []
snow_pws_data = []
snow_pwd_data = []
pws_precip_data = []
pwd_precip_data = []
pres_data = []
rh_data = []
temp_data = []
vis_pws_data =[]
vis_pwd_data = []
wdir_data = []
wspd_data = []
time_data = []
###########################################################################################################################
#Looping through the clear day dates and pulling the values that correspond to those times from the ARM/SOM Winning Node 
# NetCDF file and appending them to the empty lists that we made above. You want to store just the values and not have array[]
#so you will want to state the type the value will be int/float. 
for val in clear_days: 
    node = int(dx['Node'].loc[val].values)
    snow_pws =float(dx['pws_cumul_snow'].loc[val].values)
    snow_pwd = float(dx['pwd_cumul_snow'].loc[val].values)
    pws_precip = float(dx['pws_precip_rate_mean_1min'].loc[val].values)
    pwd_precip = float(dx['pwd_precip_rate_mean_1min'].loc[val].values)
    pres = float(dx['atmos_pressure'].loc[val].values)
    rh = float(dx['rh_mean'].loc[val].values)
    temp = float(dx['temp_mean'].loc[val].values)
    vis_pws = float(dx['pws_mean_vis_1min'].loc[val].values)
    vis_pwd = float(dx['pwd_mean_vis_1min'].loc[val].values)
    wdir = float(dx['wdir_vec_mean'].loc[val].values)
    wspd = float(dx['wspd_arith_mean'].loc[val].values)
    time = (dx['time'].loc[val].values)
    node_data.append(node)
    snow_pws_data.append(snow_pws)
    snow_pwd_data.append(snow_pwd)
    pws_precip_data.append(pws_precip)
    pwd_precip_data.append(pwd_precip)
    pres_data.append(pres)
    rh_data.append(rh)
    temp_data.append(temp)
    vis_pws_data.append(vis_pws)
    vis_pwd_data.append(vis_pwd)
    wdir_data.append(wdir)
    wspd_data.append(wspd)
    times.append(time)

In [16]:
#Generating an xarray that is of the dimension time.
clear_day = xr.Dataset(
    data_vars = dict(
        Node=(['time'], node_data), pws_cumul_snow = (['time'], snow_pws_data), pwd_cumul_snow = (['time'], snow_pwd_data), pws_precip_rate_mean_1min = (['time'],pws_precip_data),
        pwd_precip_rate_mean_1min =(['time'], pwd_precip_data), atmos_pressure = (['time'], pres_data), rh_mean =(['time'], rh_data), temp_mean = (['time'], temp_data),
        pws_mean_vis_1min=(['time'], vis_pws_data), pwd_mean_vis_1min=(['time'], vis_pwd_data), wdir_vec_mean=(['time'],wdir_data), wspd_arith_mean=(['time'], wspd_data) 
    ),
    coords=dict(
        time = (['time'],times),
       ),
    attrs=dict(
        description="ARM Data/Winning Nodes for the Clear Day data [2010-2020] provided by Lauren V."
    ),

)    

In [17]:
#Checking the NetCDF looks the way we want it to look.
clear_day

<xarray.Dataset>
Dimensions:                    (time: 3083)
Coordinates:
  * time                       (time) datetime64[ns] 2010-12-02T06:00:00 ... ...
Data variables:
    Node                       (time) int32 7 7 7 7 7 7 7 ... 19 19 19 19 19 19
    pws_cumul_snow             (time) float64 nan nan nan nan ... nan nan nan
    pwd_cumul_snow             (time) float64 492.0 492.0 492.0 ... 185.0 185.0
    pws_precip_rate_mean_1min  (time) float64 nan nan nan nan ... nan nan nan
    pwd_precip_rate_mean_1min  (time) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    atmos_pressure             (time) float64 102.2 102.3 102.3 ... 100.9 100.9
    rh_mean                    (time) float64 81.27 80.99 80.91 ... 76.96 77.19
    temp_mean                  (time) float64 -21.88 -22.13 ... -31.57 -31.64
    pws_mean_vis_1min          (time) float64 nan nan nan nan ... nan nan nan
    pwd_mean_vis_1min          (time) float64 1.999e+04 2e+04 ... 2e+04 2e+04
    wdir_vec_mean              (time) float64 347.2 354.5 215.2 ... 59.9 59.05
    wspd_arith_mean            (time) float64 6.349 5.861 5.411 ... 4.466 4.487
Attributes:
    description:  ARM Data/Winning Nodes for the Clear Day data [2010-2020] p...

In [287]:
#Saving our new dataframe so it can be used in other codes
clear_day.to_netcdf('/Users/research/thesis_code/ARM_Code_Steps/ARM_clear_data_2010_2020.nc')